In [ ]:
import pandas as pd
df = pd.read_csv('./heart_failure_clinical_records_dataset.csv') #importing dataset

In [ ]:
x = df.drop("DEATH_EVENT",axis=1) #axis needed #df.drop death event is the variable we are predicting, it is not a dependent variable it is a independent variable
y = df["DEATH_EVENT"] #what we are trying to predict

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 2)

In [ ]:
#randomforest = RandomForestClassifier(random_state=1,n_estimators=800,max_depth=20,min_samples_split=5,min_samples_leaf=1) #ideal parameters
randomforest = RandomForestClassifier()
randomforest.fit(x_train,y_train)
y_prediction = randomforest.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import classification_report
classification_report(y_test,y_prediction)

'              precision    recall  f1-score   support\n\n           0       0.93      1.00      0.97        43\n           1       1.00      0.82      0.90        17\n\n    accuracy                           0.95        60\n   macro avg       0.97      0.91      0.93        60\nweighted avg       0.95      0.95      0.95        60\n'

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
n_estimators = [100,300,500,800,1200] #returns the result of hyperparamter tuning
max_depth = [5,8,15,25,30]
min_samples_split = [2,5,10,15,100]
min_samples_leaf = [1,2,5,10]
HyperP = dict(n_estimators = n_estimators, max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf)
gridf = GridSearchCV(randomforest,HyperP,cv=3,verbose=1,n_jobs=-1)
bestf = gridf.fit(x_train,y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   34.0s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 18.0min finished


In [ ]:
import time

In [ ]:
%timeit randomforest

The slowest run took 40.67 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 34.2 ns per loop


In [ ]:
%timeit randomforest.fit

The slowest run took 58.27 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 124 ns per loop


In [ ]:
%timeit y_prediction

10000000 loops, best of 5: 29.8 ns per loop


In [ ]:
 %timeit gridf

The slowest run took 31.88 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 32.4 ns per loop


In [ ]:
%timeit bestf

The slowest run took 42.95 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 32.2 ns per loop


In [ ]:
pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-cp37-none-any.whl size=30180 sha256=729545de6447ad28e744c0fbf67b3af6d08a11ea9ff0c5b85cad550a799925d2
  Stored in directory: /root/.cache/pip/wheels/02/e4/0b/aaab481fc5dd2a4ea59e78bc7231bb6aae7635ca7ee79f8ae5
Successfully built memory-profiler


In [ ]:
%load_ext memory_profiler

In [ ]:
%memit randomforest

peak memory: 158.47 MiB, increment: 0.00 MiB


In [ ]:
%memit randomforest.fit

peak memory: 158.47 MiB, increment: 0.00 MiB


In [ ]:
%memit y_prediction

peak memory: 158.47 MiB, increment: 0.00 MiB


In [ ]:
%memit gridf

peak memory: 158.47 MiB, increment: 0.00 MiB


In [ ]:
%memit bestf

peak memory: 158.47 MiB, increment: 0.00 MiB


In [ ]:
report = classification_report(y_test,y_prediction,output_dict = True)
import seaborn as sns
sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True)

In [ ]:
from sklearn.metrics import roc_auc_score
auc_score1 = roc_auc_score(y_test, y_prediction)
print(auc_score1)

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
metrics.plot_roc_curve(randomforest, x_test, y_test)
plt.show()

In [ ]:
import numpy as np

In [ ]:
#Cost assumptions
'''Cost of FP: 1, cost of FN: 2, cost of TN: 0, cost of TP: -1'''
cost_TP = -1; cost_FP = 1; cost_FN = 2; cost_TN = 0;
cb_matrix = -1*np.array([[cost_TP, cost_FP],[cost_FN, cost_TN]])

In [ ]:
def standard_confusion_matrix(y_true, y_predict):
    TP = sum((y_true == 1) & (y_predict == 1))
    TN = sum((y_true == 0) & (y_predict == 0))
    FP = sum((y_true == 0) & (y_predict == 1))
    FN = sum((y_true == 1) & (y_predict == 0))
    
    return np.array([[TP,FP],[FN,TN]])

In [ ]:
def profit_curve(cb, predict_probas, labels):
    indices = np.argsort(predict_probas)
    sorted_probs = predict_probas[indices]
    sorted_labels = labels[indices]
    
    profit_list = []
    
    for sp, sl in zip(sorted_probs, sorted_labels):
        predict_labels = sorted_probs > sp + .0001
        conf_mat = standard_confusion_matrix(sorted_labels, predict_labels)
        profit_list.append((cb * conf_mat).sum() / float(len(labels)))
    return profit_list, sorted_probs

In [ ]:
def plot_profit_curve(model, label, costbenefit, X_train, X_test, y_train, y_test, col):
    model.fit(X_train, y_train)
    predict_probas = model.predict_proba(X_test)[:,1]
    profit_list, sorted_probs = profit_curve(costbenefit, predict_probas, y_test)
    max_index = np.argmax(profit_list)
    max_threshold = sorted_probs[max_index]
    max_profit = profit_list[max_index]
    
    plt.plot(sorted_probs, profit_list, label=label, color=col, linewidth=3)
    plt.plot(max_threshold, max_profit, '.', color=col, markersize=18)

In [ ]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [ ]:
models = [randomforest]
fig = plt.figure(figsize=(10,8))
# fig.set_facecolor('#F2F2F2')
colors = ['r', 'g', 'b', 'm']
for i, model in enumerate(models):
    plot_profit_curve(model, model.__class__.__name__, cb_matrix,
                      x_train, x_test, y_train, y_test, colors[i])
plt.title("Profit Curves")
plt.xlabel("Percentage of test instances (decreasing by score)")
plt.ylabel("Profit")
plt.legend(loc='lower right')
plt.savefig('Profit_curve.png', facecolor=fig.get_facecolor())
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test,y_prediction)
print(cf_matrix)

In [ ]:
import seaborn as sns
import numpy as np

In [ ]:
group_names = ['True Neg','false Pos','False Neg','True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf_matrix.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

https://stackoverflow.com/questions/28200786/how-to-plot-scikit-learn-classification-report
https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea
https://github.com/SydneyLauren/Profit-curves-and-xkcd-plotting
https://community.datarobot.com/t5/resources/profit-curve/ta-p/7389
https://www.analyticsvidhya.com/blog/2020/06/auc-roc-curve-machine-learning/
https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/
https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5
https://gist.github.com/fclesio/bb5871281debc07d75127552c56b08b5
https://stackoverflow.com/questions/61705257/sklearn-plotting-classification-report-gives-a-different-output-than-basic-avg
https://www.pyimagesearch.com/2016/08/15/how-to-tune-hyperparameters-with-python-and-scikit-learn/
https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
https://towardsdatascience.com/optimizing-hyperparameters-in-random-forest-classification-ec7741f9d3f6

